In [2]:
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [214]:
password = 'Password' #insert yours
db_name = 'zams'

In [215]:
conn = pymysql.connect(host='127.0.0.1', 
                       port=3306, 
                       user='root', 
                       password=password, 
                       database=db_name)

In [216]:
#Connecting to MySQL database
connection = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)

Note that result for all queries are for the first quarter of 2024

In [217]:
query1 = """
SELECT 
    d.Depart_Name,
    round(MAX(emp_avg_salaries.avg_salary), 2) AS Max_Salary,
    round(AVG(s.Gross_Pay),2) AS Department_Avg_Salary
FROM zams.salaries s
Left JOIN zams.employees e ON s.Employee_ID = e.Employee_ID
Left JOIN zams.departments d ON e.Depart_ID = d.Depart_ID
LEFT JOIN (
    SELECT 
        e.Employee_ID,
        AVG(s.Gross_Pay) AS avg_salary
    FROM zams.salaries s
    JOIN zams.employees e ON s.Employee_ID = e.Employee_ID
    GROUP BY e.Employee_ID
) AS emp_avg_salaries ON e.Employee_ID = emp_avg_salaries.Employee_ID

GROUP BY d.Depart_Name;
"""
query1 = pd.read_sql(query1, con=connection)

In [218]:
query1 #note this is for the first quarter

,Depart_Name,Max_Salary,Department_Avg_Salary
0,Communications,11600.00,10450.55
1,Engineering and Technology,12772.22,11497.92
2,Legal,12827.78,11377.08
3,Finance,18500.00,13505.36
4,Marketing,13241.66,11534.03
5,Service and Support,13533.33,10557.41
6,Design,15266.67,12447.92
7,Sales,15125.00,11654.17
8,Facilities,13177.78,10619.68


In [219]:
query2 = """
SELECT 
    d.Depart_Name AS DepartmentName,
    SUM(s.Quarterly_Bonus) AS SumOfBonuses,
    SUM(s.Gross_Pay) AS SumOfGrossSalaries,
    d.Budget_millions,
    ROUND(((SUM(s.Quarterly_Bonus) + SUM(s.Gross_Pay)) / (d.Budget_millions*1000000)), 4) * 100 AS PercentageOfBudgetToSalaries
FROM 
    Employees e
JOIN 
    Salaries s ON e.Employee_ID = s.Employee_ID
JOIN 
    Departments d ON e.Depart_ID = d.Depart_ID
GROUP BY 
    d.Depart_Name, d.Budget_millions;

"""
query2 = pd.read_sql(query2, con=connection)

In [220]:
query2

,DepartmentName,SumOfBonuses,SumOfGrossSalaries,Budget_millions,PercentageOfBudgetToSalaries
0,Communications,82000.0,156758.32,7,3.41
1,Engineering and Technology,72000.0,137974.99,25,0.84
2,Legal,105000.0,204787.49,6,5.16
3,Finance,148500.0,283612.50,15,2.88
4,Marketing,109500.0,207612.50,10,3.17
5,Service and Support,100000.0,190033.34,8,3.63
6,Design,114000.0,224062.52,9,3.76
7,Sales,73500.0,139850.01,12,1.78
8,Facilities,102500.0,191154.16,14,2.10


In [221]:
query3 = """
SELECT
    Depart_Name,
    Employee_ID,
    First_Name,
    Last_Name,
    gross_pay
FROM (
    SELECT
        d.Depart_Name,
        e.Employee_ID,
        e.First_Name,
        e.Last_Name,
        s.gross_pay,
        RANK() OVER (PARTITION BY d.Depart_ID ORDER BY s.gross_pay DESC) as salary_rank
    FROM
        zams.departments d
    JOIN
        zams.employees e ON d.Depart_ID = e.Depart_ID
    JOIN
        zams.salaries s ON e.Employee_ID = s.Employee_ID
    WHERE
        MONTH(STR_TO_DATE(s.pay_date, "%%m-%%d-%%Y")) = 3 AND
        YEAR(STR_TO_DATE(s.pay_date, "%%m-%%d-%%Y")) = 2024
) ranked_salaries
WHERE salary_rank = 1;

"""

query3 = pd.read_sql(query3, con=connection)

In [222]:
query3

,Depart_Name,Employee_ID,First_Name,Last_Name,gross_pay
0,Engineering and Technology,25,Lori,Waller,13183.33
1,Sales,39,Ashley,Riley,14500.00
2,Marketing,16,Michael,Murray,13533.33
3,Marketing,2,ikun,Cai,13533.33
4,Design,33,Angela,Smith,17466.67
5,Finance,49,Joshua,Beck,20583.33
6,Legal,12,Jennifer,Vasquez,15283.33
7,Legal,1,Jackie,Chan,15283.33
8,Communications,31,Theodore,Smith,13533.33
9,Service and Support,38,Steven,Barrera,14500.00


In [223]:
query4 = """
SELECT 
e.Employee_ID,
e.First_Name, 
e.Last_Name,
d.Depart_Name AS Department,
ROUND(AVG(s.Gross_Pay),2) AS Avg_Gross_Wage,
ROUND(AVG(s.Tax_Deduction),2) AS Avg_Tax_Paid,
ROUND(AVG(s.Net_Pay),2) AS Avg_Net_Wage

FROM salaries AS s

LEFT JOIN employees AS e
    ON s.Employee_ID = e.Employee_ID
    
LEFT JOIN departments AS d
    ON e.Depart_ID = d.Depart_ID
    
GROUP BY e.Employee_ID, e.First_Name, e.Last_Name, d.Depart_Name;
    
"""
query4 = pd.read_sql(query4, con=connection)

In [224]:
query4.sort_values('Employee_ID')

,Employee_ID,First_Name,Last_Name,Department,Avg_Gross_Wage,Avg_Tax_Paid,Avg_Net_Wage
15,1,Jackie,Chan,Legal,12183.33,4069.23,7358.73
16,2,ikun,Cai,Marketing,12388.89,3840.55,7780.22
17,3,Dennis,Hancock,Sales,12648.61,2592.97,9271.43
18,4,Jennifer,Gibson,Design,15266.67,3099.13,11221.00
19,5,Angela,Ellison,Marketing,9479.17,2407.71,6483.75
22,6,Aaron,Kemp,Finance,8500.00,2380.00,5593.00
24,7,Daniel,Farley,Finance,10312.50,1959.37,7713.75
26,8,Barbara,Adams,Legal,12100.00,4755.30,6594.50
28,9,Nicholas,Mckinney,Finance,15405.56,4221.12,10229.29
2,10,Joshua,King,Legal,8644.44,1780.76,6327.73


In [225]:
query5 = """
SELECT 
    d.Depart_Name,
    COUNT(DISTINCT CASE WHEN s.Total_Work_Time > 160 THEN s.Employee_ID END) AS Employees_Working_Overtime,
    COUNT(DISTINCT e.Employee_ID) AS Total_Employees
FROM 
    zams.salaries s
LEFT JOIN 
    zams.employees e ON s.Employee_ID = e.Employee_ID
LEFT JOIN 
    zams.departments d ON e.Depart_ID = d.Depart_ID
GROUP BY 
    d.Depart_Name;

"""
query5 = pd.read_sql(query5, con=connection)

In [226]:
query5

,Depart_Name,Employees_Working_Overtime,Total_Employees
0,Communications,1,5
1,Design,4,6
2,Engineering and Technology,3,4
3,Facilities,3,6
4,Finance,5,7
5,Legal,5,6
6,Marketing,3,6
7,Sales,2,4
8,Service and Support,3,6


In [227]:
query6 = """
SELECT 
    e.First_Name AS Employee_Name,
    MONTH(e.Birth_Date) AS Birth_Month,
    DAY(e.Birth_Date) AS Birth_Day,
    d.Depart_Name AS Department
FROM 
    Employees AS e
JOIN 
    Departments AS d
ON 
    e.Depart_ID = d.Depart_ID
WHERE 
    MONTH(e.Birth_Date)> 6
ORDER BY Birth_Month ASC, Birth_Day ASC
"""
query6 = pd.read_sql(query6, con=connection)

In [228]:
query6

,Employee_Name,Birth_Month,Birth_Day,Department
0,Amanda,7,23,Service and Support
1,Michael,7,24,Marketing
2,ikun,7,30,Marketing
3,Nicole,7,31,Legal
4,Joshua,8,31,Legal
5,Dennis,9,3,Sales
6,Samantha,9,3,Finance
7,Monica,9,7,Facilities
8,Michael,9,24,Communications
9,Megan,9,24,Service and Support
